## 필요 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import requests
import json
from tqdm import tqdm
import time

## 데이터셋 불러오기

In [2]:
med = pd.read_csv('C:/Users/user/Downloads/1. 병원정보서비스 2021.6.csv', encoding = 'cp949')

med.head(6)

,암호화YKIHO코드,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,우편번호,...,치과일반의 인원수,치과인턴 인원수,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,x좌표,y좌표
0,JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,가톨릭대학교인천성모병원,1,상급종합,220000,인천,220003,인천부평구,NaN,21431,...,1,0,0,2,0,0,0,0,126.724899,37.484831
1,JDQ4MTYyMiM1MSMkMSMkNCMkODkkMzgxMzUxIzExIyQxIy...,강릉아산병원,1,상급종합,320000,강원,320100,강릉시,NaN,25440,...,0,0,0,2,0,0,0,0,128.857841,37.818433
2,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,강북삼성병원,1,상급종합,110000,서울,110016,종로구,NaN,3181,...,3,0,0,4,0,0,0,0,126.967750,37.568408
3,JDQ4MTg4MSM1MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,건국대학교병원,1,상급종합,110000,서울,110023,광진구,NaN,5030,...,0,0,0,3,0,0,0,0,127.071828,37.540376
4,JDQ4MTYyMiM4MSMkMSMkMCMkODkkMzgxMzUxIzExIyQxIy...,경북대학교병원,1,상급종합,230000,대구,230006,대구중구,NaN,41944,...,0,0,0,1,0,0,0,0,128.604125,35.866774
5,JDQ4MTYyMiM4MSMkMSMkNCMkODkkMzgxMzUxIzExIyQxIy...,경상국립대학교병원,1,상급종합,380000,경남,380500,진주시,NaN,52727,...,1,0,0,1,0,0,0,0,128.095672,35.176325


In [3]:
dj = med[med['시도코드명'] == '대전']
dj.head(3)

,암호화YKIHO코드,요양기관명,종별코드,종별코드명,시도코드,시도코드명,시군구코드,시군구코드명,읍면동,우편번호,...,치과일반의 인원수,치과인턴 인원수,치과레지던트 인원수,치과전문의 인원수,한방일반의 인원수,한방인턴 인원수,한방레지던트 인원수,한방전문의 인원수,x좌표,y좌표
34,JDQ4MTYyMiM2MSMkMSMkMiMkODkkMzgxMzUxIzExIyQxIy...,충남대학교병원,1,상급종합,250000,대전,250005,대전중구,NaN,35015,...,0,0,0,3,0,0,0,0,127.415363,36.316908
98,JDQ4MTYyMiM2MSMkMSMkMiMkODkkMzgxMzUxIzExIyQyIy...,근로복지공단 대전병원,11,종합병원,250000,대전,250002,대전대덕구,NaN,34384,...,1,0,0,0,0,0,0,0,127.428549,36.368398
118,JDQ4MTYyMiM2MSMkMSMkMiMkODkkMzgxMzUxIzExIyQxIy...,대전보훈병원,11,종합병원,250000,대전,250002,대전대덕구,NaN,34314,...,4,0,0,2,1,0,0,0,127.439694,36.447316


In [4]:
dj.shape

(2304, 29)

In [5]:
dj['시군구코드명'].unique()

array(['대전중구', '대전대덕구', '대전동구', '대전서구', '대전유성구'], dtype=object)

In [6]:
dj.columns

Index(['암호화YKIHO코드', '요양기관명', '종별코드', '종별코드명', '시도코드', '시도코드명', '시군구코드',
       '시군구코드명', '읍면동', '우편번호', '주소', '전화번호', '병원URL', '개설일자', '총의사수',
       '의과일반의 인원수', '의과인턴 인원수', '의과레지던트 인원수', '의과전문의 인원수', '치과일반의 인원수',
       '치과인턴 인원수', '치과레지던트 인원수', '치과전문의 인원수', '한방일반의 인원수', '한방인턴 인원수',
       '한방레지던트 인원수', '한방전문의 인원수', 'x좌표', 'y좌표'],
      dtype='object')

## API + API 사용을 위한 데이터 정제

In [7]:
rest_api_key = 'ad05672253229712c2490cce54ba564c'

In [8]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [9]:
kakao = KakaoLocalAPI(rest_api_key)

In [10]:
# 불필요한 열 제거
dj = dj[['요양기관명', '종별코드명', '시군구코드명', '주소']]
dj

,요양기관명,종별코드명,시군구코드명,주소
34,충남대학교병원,상급종합,대전중구,대전광역시 중구 문화로 282 (대사동)
98,근로복지공단 대전병원,종합병원,대전대덕구,대전광역시 대덕구 계족로 637 (법동)
118,대전보훈병원,종합병원,대전대덕구,대전광역시 대덕구 대청로82번길 147 147
119,대전한국병원,종합병원,대전동구,대전광역시 동구 동서대로 1672 (성남동)
121,대청병원,종합병원,대전서구,대전광역시 서구 계백로 1322 (정림동)
...,...,...,...,...
74220,회생한의원,한의원,대전서구,"대전광역시 서구 월평동로 83 308,310호 (월평동, 다모아아파트상가)"
74263,효자한의원,한의원,대전유성구,대전광역시 유성구 진잠로42번길 117 (원내동)
74290,후한의원,한의원,대전서구,대전광역시 서구 둔산남로 101 (둔산동)
74326,휴한의원,한의원,대전서구,대전광역시 서구 둔산로123번길 15 (둔산동)


In [11]:
dj.isnull().sum()

요양기관명     0
종별코드명     0
시군구코드명    0
주소        0
dtype: int64

In [12]:
dj.reset_index(drop = True, inplace = True)
dj.shape

(2304, 4)

In [13]:
dj[['add_1', 'add_2', 'add_3', 'add_4', 'add_rest']] = pd.DataFrame(dj.주소.str.split(' ', 4).tolist())
dj

,요양기관명,종별코드명,시군구코드명,주소,add_1,add_2,add_3,add_4,add_rest
0,충남대학교병원,상급종합,대전중구,대전광역시 중구 문화로 282 (대사동),대전광역시,중구,문화로,282,(대사동)
1,근로복지공단 대전병원,종합병원,대전대덕구,대전광역시 대덕구 계족로 637 (법동),대전광역시,대덕구,계족로,637,(법동)
2,대전보훈병원,종합병원,대전대덕구,대전광역시 대덕구 대청로82번길 147 147,대전광역시,대덕구,대청로82번길,147,147
3,대전한국병원,종합병원,대전동구,대전광역시 동구 동서대로 1672 (성남동),대전광역시,동구,동서대로,1672,(성남동)
4,대청병원,종합병원,대전서구,대전광역시 서구 계백로 1322 (정림동),대전광역시,서구,계백로,1322,(정림동)
...,...,...,...,...,...,...,...,...,...
2299,회생한의원,한의원,대전서구,"대전광역시 서구 월평동로 83 308,310호 (월평동, 다모아아파트상가)",대전광역시,서구,월평동로,83,"308,310호 (월평동, 다모아아파트상가)"
2300,효자한의원,한의원,대전유성구,대전광역시 유성구 진잠로42번길 117 (원내동),대전광역시,유성구,진잠로42번길,117,(원내동)
2301,후한의원,한의원,대전서구,대전광역시 서구 둔산남로 101 (둔산동),대전광역시,서구,둔산남로,101,(둔산동)
2302,휴한의원,한의원,대전서구,대전광역시 서구 둔산로123번길 15 (둔산동),대전광역시,서구,둔산로123번길,15,(둔산동)


In [14]:
dj.drop(['add_1', 'add_2', 'add_rest'], axis = 1, inplace = True)
dj

,요양기관명,종별코드명,시군구코드명,주소,add_3,add_4
0,충남대학교병원,상급종합,대전중구,대전광역시 중구 문화로 282 (대사동),문화로,282
1,근로복지공단 대전병원,종합병원,대전대덕구,대전광역시 대덕구 계족로 637 (법동),계족로,637
2,대전보훈병원,종합병원,대전대덕구,대전광역시 대덕구 대청로82번길 147 147,대청로82번길,147
3,대전한국병원,종합병원,대전동구,대전광역시 동구 동서대로 1672 (성남동),동서대로,1672
4,대청병원,종합병원,대전서구,대전광역시 서구 계백로 1322 (정림동),계백로,1322
...,...,...,...,...,...,...
2299,회생한의원,한의원,대전서구,"대전광역시 서구 월평동로 83 308,310호 (월평동, 다모아아파트상가)",월평동로,83
2300,효자한의원,한의원,대전유성구,대전광역시 유성구 진잠로42번길 117 (원내동),진잠로42번길,117
2301,후한의원,한의원,대전서구,대전광역시 서구 둔산남로 101 (둔산동),둔산남로,101
2302,휴한의원,한의원,대전서구,대전광역시 서구 둔산로123번길 15 (둔산동),둔산로123번길,15


In [15]:
dj['add_4'] = dj['add_4'].str.replace(',', '')
dj

,요양기관명,종별코드명,시군구코드명,주소,add_3,add_4
0,충남대학교병원,상급종합,대전중구,대전광역시 중구 문화로 282 (대사동),문화로,282
1,근로복지공단 대전병원,종합병원,대전대덕구,대전광역시 대덕구 계족로 637 (법동),계족로,637
2,대전보훈병원,종합병원,대전대덕구,대전광역시 대덕구 대청로82번길 147 147,대청로82번길,147
3,대전한국병원,종합병원,대전동구,대전광역시 동구 동서대로 1672 (성남동),동서대로,1672
4,대청병원,종합병원,대전서구,대전광역시 서구 계백로 1322 (정림동),계백로,1322
...,...,...,...,...,...,...
2299,회생한의원,한의원,대전서구,"대전광역시 서구 월평동로 83 308,310호 (월평동, 다모아아파트상가)",월평동로,83
2300,효자한의원,한의원,대전유성구,대전광역시 유성구 진잠로42번길 117 (원내동),진잠로42번길,117
2301,후한의원,한의원,대전서구,대전광역시 서구 둔산남로 101 (둔산동),둔산남로,101
2302,휴한의원,한의원,대전서구,대전광역시 서구 둔산로123번길 15 (둔산동),둔산로123번길,15


In [16]:
dj['주소'] = dj['add_3'] + ' ' + dj['add_4']
dj

,요양기관명,종별코드명,시군구코드명,주소,add_3,add_4
0,충남대학교병원,상급종합,대전중구,문화로 282,문화로,282
1,근로복지공단 대전병원,종합병원,대전대덕구,계족로 637,계족로,637
2,대전보훈병원,종합병원,대전대덕구,대청로82번길 147,대청로82번길,147
3,대전한국병원,종합병원,대전동구,동서대로 1672,동서대로,1672
4,대청병원,종합병원,대전서구,계백로 1322,계백로,1322
...,...,...,...,...,...,...
2299,회생한의원,한의원,대전서구,월평동로 83,월평동로,83
2300,효자한의원,한의원,대전유성구,진잠로42번길 117,진잠로42번길,117
2301,후한의원,한의원,대전서구,둔산남로 101,둔산남로,101
2302,휴한의원,한의원,대전서구,둔산로123번길 15,둔산로123번길,15


In [17]:
dj.drop(['add_3', 'add_4'], axis = 1, inplace = True)
dj

,요양기관명,종별코드명,시군구코드명,주소
0,충남대학교병원,상급종합,대전중구,문화로 282
1,근로복지공단 대전병원,종합병원,대전대덕구,계족로 637
2,대전보훈병원,종합병원,대전대덕구,대청로82번길 147
3,대전한국병원,종합병원,대전동구,동서대로 1672
4,대청병원,종합병원,대전서구,계백로 1322
...,...,...,...,...
2299,회생한의원,한의원,대전서구,월평동로 83
2300,효자한의원,한의원,대전유성구,진잠로42번길 117
2301,후한의원,한의원,대전서구,둔산남로 101
2302,휴한의원,한의원,대전서구,둔산로123번길 15


## 행정동, 위경도 추출

In [18]:
add_list = dj['주소']
add_list

0           문화로 282
1           계족로 637
2       대청로82번길 147
3         동서대로 1672
4          계백로 1322
           ...     
2299        월평동로 83
2300    진잠로42번길 117
2301       둔산남로 101
2302    둔산로123번길 15
2303     계백로 1554-1
Name: 주소, Length: 2304, dtype: object

In [19]:
# api 호출 확인
kakao.search_address(add_list[0])

{'documents': [{'address': {'address_name': '충북 진천군 진천읍 성석리 512-9',
    'b_code': '4375025025',
    'h_code': '4375025000',
    'main_address_no': '512',
    'mountain_yn': 'N',
    'region_1depth_name': '충북',
    'region_2depth_name': '진천군',
    'region_3depth_h_name': '진천읍',
    'region_3depth_name': '진천읍 성석리',
    'sub_address_no': '9',
    'x': '127.44233599242',
    'y': '36.863143498594'},
   'address_name': '충북 진천군 진천읍 문화로 282',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '충북 진천군 진천읍 문화로 282',
    'building_name': '',
    'main_building_no': '282',
    'region_1depth_name': '충북',
    'region_2depth_name': '진천군',
    'region_3depth_name': '진천읍 성석리',
    'road_name': '문화로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.44233599242',
    'y': '36.863143498594',
    'zone_no': '27849'},
   'x': '127.44233599242',
   'y': '36.863143498594'},
  {'address': {'address_name': '대전 중구 대사동 640',
    'b_code': '3014011000',
    'h_code': '3014063

In [20]:
dj['행정동'] = np.nan
dj['위도'] = np.nan
dj['경도'] = np.nan
dj

,요양기관명,종별코드명,시군구코드명,주소,행정동,위도,경도
0,충남대학교병원,상급종합,대전중구,문화로 282,NaN,NaN,NaN
1,근로복지공단 대전병원,종합병원,대전대덕구,계족로 637,NaN,NaN,NaN
2,대전보훈병원,종합병원,대전대덕구,대청로82번길 147,NaN,NaN,NaN
3,대전한국병원,종합병원,대전동구,동서대로 1672,NaN,NaN,NaN
4,대청병원,종합병원,대전서구,계백로 1322,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2299,회생한의원,한의원,대전서구,월평동로 83,NaN,NaN,NaN
2300,효자한의원,한의원,대전유성구,진잠로42번길 117,NaN,NaN,NaN
2301,후한의원,한의원,대전서구,둔산남로 101,NaN,NaN,NaN
2302,휴한의원,한의원,대전서구,둔산로123번길 15,NaN,NaN,NaN


In [21]:
dj_new = dj.copy()

In [22]:
for i in tqdm(range(len(add_list))):
    try :
        dj_new['행정동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_h_name']
        dj_new['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        dj_new['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass
    
dj_new

  0%|          | 0/2304 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_3044\722878745.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj_new['행정동'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['address']['region_3depth_h_name']
C:\Users\user\AppData\Local\Temp\ipykernel_3044\722878745.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj_new['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
C:\Users\user\AppData\Local\Temp\ipykernel_3044\722878745.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the 

,요양기관명,종별코드명,시군구코드명,주소,행정동,위도,경도
0,충남대학교병원,상급종합,대전중구,문화로 282,진천읍,36.863143498594,127.44233599242
1,근로복지공단 대전병원,종합병원,대전대덕구,계족로 637,법1동,36.3692226389729,127.427893917601
2,대전보훈병원,종합병원,대전대덕구,대청로82번길 147,신탄진동,36.447316412604,127.439693956141
3,대전한국병원,종합병원,대전동구,동서대로 1672,성남동,36.3481758122459,127.43583507554
4,대청병원,종합병원,대전서구,계백로 1322,정림동,36.3082267126412,127.370510687283
...,...,...,...,...,...,...,...
2299,회생한의원,한의원,대전서구,월평동로 83,월평3동,36.3611708878133,127.366305404613
2300,효자한의원,한의원,대전유성구,진잠로42번길 117,진잠동,36.2989299146086,127.319923688063
2301,후한의원,한의원,대전서구,둔산남로 101,둔산1동,36.3493130580634,127.388807536131
2302,휴한의원,한의원,대전서구,둔산로123번길 15,둔산1동,36.3519974722872,127.387421738653


In [23]:
dj_new.isnull().sum()

요양기관명      0
종별코드명      0
시군구코드명     0
주소         0
행정동       15
위도        15
경도        15
dtype: int64

In [33]:
dj_null = dj_new[dj_new['행정동'].isnull()]
dj_null

,요양기관명,종별코드명,시군구코드명,주소,행정동,위도,경도
12,국군대전병원,병원,대전유성구,자운로 90-0,NaN,NaN,NaN
285,대전교도소부속의원,의원,대전유성구,한우물로66번길 6,NaN,NaN,NaN
294,대전소년원부속의원,의원,대전동구,산내로 1398-41,NaN,NaN,NaN
444,민가정의학과의원,의원,대전동구,산내로,NaN,NaN,NaN
935,인제정형외과의원,의원,대전서구,계백로 1264-1264,NaN,NaN,NaN
1087,튼튼한정형외과의원,의원,대전유성구,관평2로 7,NaN,NaN,NaN
1217,국군대전치과병원,치과병원,대전유성구,자운로 90,NaN,NaN,NaN
1276,김응학치과의원,치과의원,대전유성구,관들1길 63-63,NaN,NaN,NaN
1312,대전교도소부속치과의원,치과의원,대전유성구,한우물로66번길 6,NaN,NaN,NaN
1500,시티치과의원,치과의원,대전서구,둔산로 76-76,NaN,NaN,NaN


In [34]:
dj_null.shape

(15, 7)

In [35]:
# 주소 미확인, 총 15건

# 카카오맵 상 확인되는 주소로 수정, 11건
dj_null.loc[12]['주소'] = '자운동 97'
dj_null.loc[294]['주소'] = '대성동 295-1'
dj_null.loc[444]['주소'] = '산내로 1317'
dj_null.loc[935]['주소'] = '계백로 1264'
dj_null.loc[1087]['주소'] = '관평2로 7-7'
dj_null.loc[1276]['주소'] = '관들1길 63'
dj_null.loc[1500]['주소'] = '둔산로 76'
dj_null.loc[1637]['주소'] = '대종로 422'
dj_null.loc[2165]['주소'] = '월드컵대로275번길 39 2층'
dj_null.loc[2208]['주소'] = '관평2로 22 3층'
dj_null.loc[2277]['주소'] = '옥천로 168 2층'

# 주소 확인 불가, 4건
# index 285, 1217, 1312, 1843
dj_null.drop([285, 1217, 1312, 1843], inplace = True)

C:\Users\user\AppData\Local\Temp\ipykernel_3044\1774295293.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj_null.drop([285, 1217, 1312, 1843], inplace = True)


In [39]:
dj_null.reset_index(drop = True, inplace = True)
dj_null

,요양기관명,종별코드명,시군구코드명,주소,행정동,위도,경도
0,국군대전병원,병원,대전유성구,자운동 97,NaN,NaN,NaN
1,대전소년원부속의원,의원,대전동구,대성동 295-1,NaN,NaN,NaN
2,민가정의학과의원,의원,대전동구,산내로 1317,NaN,NaN,NaN
3,인제정형외과의원,의원,대전서구,계백로 1264,NaN,NaN,NaN
4,튼튼한정형외과의원,의원,대전유성구,관평2로 7-7,NaN,NaN,NaN
5,김응학치과의원,치과의원,대전유성구,관들1길 63,NaN,NaN,NaN
6,시티치과의원,치과의원,대전서구,둔산로 76,NaN,NaN,NaN
7,자이부부치과의원,치과의원,대전중구,대종로 422,NaN,NaN,NaN
8,일신당한의원,한의원,대전유성구,월드컵대로275번길 39 2층,NaN,NaN,NaN
9,참솔한의원,한의원,대전동구,관평2로 22 3층,NaN,NaN,NaN


In [40]:
dj_null.shape

(11, 7)

In [41]:
add_list_1 = dj_null['주소']
add_list_1

0               자운동 97
1            대성동 295-1
2             산내로 1317
3             계백로 1264
4             관평2로 7-7
5              관들1길 63
6               둔산로 76
7              대종로 422
8     월드컵대로275번길 39 2층
9           관평2로 22 3층
10          옥천로 168 2층
Name: 주소, dtype: object

In [42]:
# api 호출 확인
kakao.search_address(add_list_1[0])

{'documents': [{'address': {'address_name': '대전 유성구 자운동 97',
    'b_code': '3020013400',
    'h_code': '3020055000',
    'main_address_no': '97',
    'mountain_yn': 'N',
    'region_1depth_name': '대전',
    'region_2depth_name': '유성구',
    'region_3depth_h_name': '신성동',
    'region_3depth_name': '자운동',
    'sub_address_no': '',
    'x': '127.349684367184',
    'y': '36.3989854472804'},
   'address_name': '대전 유성구 자운동 97',
   'address_type': 'REGION_ADDR',
   'road_address': {'address_name': '대전 유성구 자운로 90',
    'building_name': '',
    'main_building_no': '90',
    'region_1depth_name': '대전',
    'region_2depth_name': '유성구',
    'region_3depth_name': '자운동',
    'road_name': '자운로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '',
    'y': '',
    'zone_no': '34059'},
   'x': '127.349684367184',
   'y': '36.3989854472804'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [43]:
for i in tqdm(range(len(add_list_1))):
    try :
        dj_null['행정동'].iloc[i] = kakao.search_address(add_list_1[i])['documents'][0]['address']['region_3depth_h_name']
        dj_null['경도'].iloc[i] = kakao.search_address(add_list_1[i])['documents'][0]['x']
        dj_null['위도'].iloc[i] = kakao.search_address(add_list_1[i])['documents'][0]['y']
    except:
        pass
    
dj_null

  0%|          | 0/11 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_3044\4000314315.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj_null['행정동'].iloc[i] = kakao.search_address(add_list_1[i])['documents'][0]['address']['region_3depth_h_name']
C:\Users\user\AppData\Local\Temp\ipykernel_3044\4000314315.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dj_null['경도'].iloc[i] = kakao.search_address(add_list_1[i])['documents'][0]['x']
C:\Users\user\AppData\Local\Temp\ipykernel_3044\4000314315.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats 

,요양기관명,종별코드명,시군구코드명,주소,행정동,위도,경도
0,국군대전병원,병원,대전유성구,자운동 97,신성동,36.3989854472804,127.349684367184
1,대전소년원부속의원,의원,대전동구,대성동 295-1,산내동,36.2882619514665,127.466881869458
2,민가정의학과의원,의원,대전동구,산내로 1317,산내동,36.2810553816446,127.467458355727
3,인제정형외과의원,의원,대전서구,계백로 1264,정림동,36.3070005703784,127.363285146856
4,튼튼한정형외과의원,의원,대전유성구,관평2로 7-7,관평동,36.4254988842707,127.39170340758
5,김응학치과의원,치과의원,대전유성구,관들1길 63,관평동,36.4251101124103,127.389032793334
6,시티치과의원,치과의원,대전서구,둔산로 76,둔산2동,36.3510295052366,127.382249781499
7,자이부부치과의원,치과의원,대전중구,대종로 422,대흥동,36.322605814029,127.429650391479
8,일신당한의원,한의원,대전유성구,월드컵대로275번길 39 2층,온천1동,36.3504071440796,127.331400880614
9,참솔한의원,한의원,대전동구,관평2로 22 3층,관평동,36.4246701352035,127.389815402553


In [44]:
dj_new.dropna(axis = 0, inplace = True)
dj_new.shape

(2289, 7)

In [45]:
dj_total = pd.concat([dj_new, dj_null])
dj_total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2300 entries, 0 to 10
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   요양기관명   2300 non-null   object
 1   종별코드명   2300 non-null   object
 2   시군구코드명  2300 non-null   object
 3   주소      2300 non-null   object
 4   행정동     2300 non-null   object
 5   위도      2300 non-null   object
 6   경도      2300 non-null   object
dtypes: object(7)
memory usage: 143.8+ KB


In [46]:
dj_total.reset_index(drop = True, inplace = True)
dj_total.head(2)

,요양기관명,종별코드명,시군구코드명,주소,행정동,위도,경도
0,충남대학교병원,상급종합,대전중구,문화로 282,진천읍,36.863143498594,127.44233599242
1,근로복지공단 대전병원,종합병원,대전대덕구,계족로 637,법1동,36.3692226389729,127.427893917601


## 데이터셋 csv 저장

In [ ]:
dj_total.to_csv('병원정보서비스_안지인_법정동명,위경도추출.csv', index = False, encoding = 'cp949')